Oppgave 1.7: Hvor mange elbiler ble førstegangsregistrert i 2022
    Leser fra preppet parquet fil laget i oppg. 1.6

In [44]:
import polars as pl

kp = pl.read_parquet('kjoretoy_preppet.parquet')

df_ant_el_2022 = kp.filter(
    (pl.col('tekn_reg_f_g_n').cast(str).str.contains('2022')) 
    & (pl.col('Er_Elbil') == True)
)

print(f"Antall Elbiler registrert i 2022 er: {df_ant_el_2022.height}")

Antall Elbiler registrert i 2022 er: 37249


Oppgave 1.8: Hvor mange prosent av persbilene solgt i 2022 var elbiler
    Her leser vi fra tekn_reg_eier_dato og sjekker at datoen starter med året 2022, da det er slik dataen er strukturert.

In [42]:
antallElbil = len(kp.filter((pl.col('tekn_reg_eier_dato').cast(str).str.starts_with('2022')) & (pl.col('Er_Elbil') == True)))

antallBilerTotalt = len(kp.filter(pl.col('tekn_reg_eier_dato').cast(str).str.starts_with('2022')))

prosent =((antallElbil / antallBilerTotalt) * 100).__round__(2)

print(f"Antall Elbiler registrert på ny eier i 2022: {antallElbil}")
print(f"Antall alle biler registrert på ny eier i 2022: {antallBilerTotalt}")
print(f"Prosent som er elbil: {prosent}")

Antall Elbiler registrert på ny eier i 2022: 38670
Antall alle biler registrert på ny eier i 2022: 133994
Prosent som er elbil: 28.86


Oppg 1.9: Hvilken bilmodell var mest populær i 2022
    Jeg tolker denne oppgaven til å bety mest populær å kjøpe, ergo vi ser på førstegangsregistrerte biler. 
    Vi filtrerer ut alle biler fra 2022, og teller antall hendelser av modellen. Deretter sorterer vi listen etter count, synkende.

In [46]:
biler2022 = kp.filter(pl.col('tekn_reg_f_g_n').cast(str).str.starts_with('2022') & (pl.col('tekn_kjtgrp') == 101))

populæreBiler = biler2022.group_by("tekn_modell").count()

populæreBiler.sort(pl.col('count'), descending=True)

tekn_modell,count
str,u32
"""Model Y""",8719
"""ID.4 GTX 220 K…",4021
"""XC40""",3001
"""IONIQ5""",2890
"""ENYAQ 80X""",2799
"""Polestar 2""",2413
"""Mustang Mach-E…",2389
"""RAV4""",2009
"""E TRON 55""",1975


Oppg 1.10: Hvor mange gule kjøretøy ble solgt i mai 2022?